# ImportiImporting and preparing rental apartments dataeren

## Libraries and settings

In [40]:
# Libraries
import os
import re
import time
import fnmatch
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print(os.getcwd())

/workspaces/groceries/preparation


## CSV Letzte Zeile

In [41]:
import os
import pandas as pd

def create_prepared_csv(input_path, output_path, supermarket):
    # Überprüfen, ob die Ausgabedatei bereits existiert
    if not os.path.exists(output_path):
        # DataFrame aus der Eingabedatei erstellen
        df = pd.read_csv(input_path, sep=';')
        # Neue Spalte "Supermarkt" hinzufügen
        df['Supermarkt'] = supermarket
        # DataFrame in eine neue CSV-Datei schreiben
        df.to_csv(output_path, index=False)
        print(f"Die Datei {output_path} wurde erfolgreich erstellt.")
    else:
        print(f"Die Datei {output_path} existiert bereits.")

# Aldi Flour
create_prepared_csv('../CSV/Aldi_Flour.csv', '../CSV/prepared/prepared_CSV_Aldi_Flour.csv', 'Aldi')

# Coop Flour
create_prepared_csv('../CSV/Coop_Flour.csv', '../CSV/prepared/prepared_CSV_Coop_Flour.csv', 'Coop')

# Migros Flour
create_prepared_csv('../CSV/Migros_Flour.csv', '../CSV/prepared/prepared_CSV_Migros_Flour.csv', 'Migros')


Die Datei ../CSV/prepared/prepared_CSV_Aldi_Flour.csv existiert bereits.
Die Datei ../CSV/prepared/prepared_CSV_Coop_Flour.csv existiert bereits.
Die Datei ../CSV/prepared/prepared_CSV_Migros_Flour.csv existiert bereits.


## Merge Flour

In [42]:
import pandas as pd
import os

# Liste der Dateipfade
Flours_merge = [
    '../CSV/prepared/prepared_CSV_Aldi_Flour.csv',
    '../CSV/prepared/prepared_CSV_Coop_Flour.csv',
    '../CSV/prepared/prepared_CSV_Migros_Flour.csv'
]

# Eine leere DataFrame erstellen, um die Daten zu speichern
gesamt_df = pd.DataFrame()

# Durchlaufen Sie die Liste der Dateipfade und fügen Sie die Daten dem Gesamt-DataFrame hinzu
for Flour_merge in Flours_merge:
    df = pd.read_csv(Flour_merge, sep=',', encoding='utf-8')  # Änderung hier: sep=';'
    gesamt_df = pd.concat([gesamt_df, df], ignore_index=True)  # Änderung hier: Verwendung von pd.concat

# Den Gesamt-DataFrame in eine neue CSV-Datei schreiben
gesamt_df.to_csv('Flour_merge.csv', index=False)


## Importing data

In [43]:
import pandas as pd
 
# CSV-Datei einlesen
df = pd.read_csv('Flour_merge.csv')
 
# Funktion zur Berechnung des Preises pro Kilogramm
def calculate_price_per_kg(row):
    weight = row['weight_raw']
    price = row['price_raw']
    # Überprüfen, ob der Preis ein String ist und ggf. ersetzen
    if isinstance(price, str):
        price = float(price.replace(',', '.'))
    if 'x' in weight:
        quantity, unit = weight.split('x')
        quantity = int(quantity.strip())
        return price / quantity
    else:
        return price
 
# Anwenden der Funktionen
df['price_per_kg'] = df.apply(calculate_price_per_kg, axis=1)
df['weight_raw'] = '1'  # Setzt das Gewicht auf 1 kg für alle Einträge
 
# Ergebnis in eine neue CSV-Datei schreiben
df.to_csv('Flour_merge.csv', index=False)

# Show first records of data frame
df.head(70)

,product_type,web-scraper-order,web-scraper-start-url,productname_raw,price_raw,weight_raw,Supermarkt,price_per_kg
0,flour,1698753392-1,https://www.aldi-now.ch/de/vorr%C3%A4te/backar...,NATURE SUISSE BIO Halbweissmehl,2.99,1,Aldi,2.99
1,flour,1698753392-2,https://www.aldi-now.ch/de/vorr%C3%A4te/backar...,HAPPY HARVEST Zopfmehl,1.99,1,Aldi,1.99
2,flour,1698753392-3,https://www.aldi-now.ch/de/vorr%C3%A4te/backar...,HAPPY HARVEST Ruchmehl,2.19,1,Aldi,2.19
3,flour,1698753392-4,https://www.aldi-now.ch/de/vorr%C3%A4te/backar...,HAPPY HARVEST Weissmehl,0.99,1,Aldi,0.99
4,flour,1698753392-5,https://www.aldi-now.ch/de/vorr%C3%A4te/backar...,"HAPPY HARVEST Brotbackmischung, Vollkornbrot",1.99,1,Aldi,1.99
...,...,...,...,...,...,...,...,...
62,flour,1698741806-33,https://www.migros.ch/de/search?query=Mehl,Aus der Region Zopfmehl,2.50,1,Migros,2.50
63,flour,1698741806-34,https://www.migros.ch/de/search?query=Mehl,Aus der Region Zopfmehl,2.80,1,Migros,2.80
64,flour,1698741806-42,https://www.migros.ch/de/search?query=Mehl,Mehlmischung glutenfrei,4.90,1,Migros,4.90
65,flour,1698741806-43,https://www.migros.ch/de/search?query=Mehl,Aus der Region Zopfmehl,3.95,1,Migros,3.95


## Count number of rows and columns in the data frame

In [44]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

Dimension: (67, 8)
Number of rows: 67
Number of columns: 8


## Get data types (raw-format from web scraping)

In [45]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

product_type              object
web-scraper-order         object
web-scraper-start-url     object
productname_raw           object
price_raw                float64
weight_raw                object
Supermarkt                object
price_per_kg             float64
dtype: object